# Place for experimenting the progressive design

In [2]:
import yaml
import inspect
import importlib
import os,sys

sys.path.append('..')

import model_discovery.utils as U
from model_discovery.configs.gam_config import GAMConfig, GAMConfig_14M
# from model_discovery.evolution import BuildEvolution
import model_discovery.agents.flow.gau_utils
importlib.reload(model_discovery.agents.flow.gau_utils)
from model_discovery.agents.flow.gau_utils import check_and_reformat_gau_code
from model_discovery.model.composer import GAUBase, GAUTree, check_tree_name, GABComposer
from model_discovery.model.utils.modules import GABBase

from model_discovery.evolution import BuildEvolution

ckpt_dir = os.environ['CKPT_DIR']
lib_dir = U.pjoin(ckpt_dir, 'test_composer', 'lib')
test_tree = GAUTree('TestTree', None, None, None,None,lib_dir)

prompts_dir='../model_discovery/agents/prompts/'
gab_py = U.read_file(U.pjoin(prompts_dir,'gab_template.py'))
gam_prompt_path = os.path.join(prompts_dir,'gam_prompt.py')
gau_template_path = os.path.join(prompts_dir,'gau_template.py')
GAM_TEMPLATE=open(gam_prompt_path).read()
GAU_TEMPLATE=open(gau_template_path).read()
GAU_BASE=inspect.getsource(GAUBase)

GAB_BASE=inspect.getsource(GABBase)
GAB_COMPOSER=inspect.getsource(GABComposer)

c:\Users\ChengJunyan1\anaconda3\envs\modis\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\ChengJunyan1\.cache\huggingface\token
Login successful


c:\ChengJunyan1\Research\model_discovery\notebooks\..\model_discovery\agents\prompts\prompts.py:1471: SyntaxWarning: invalid escape sequence '\('
  GUE_DESIGN_PROPOSER_SYSTEM_prompt = """
c:\ChengJunyan1\Research\model_discovery\notebooks\..\model_discovery\agents\prompts\prompts.py:1696: SyntaxWarning: invalid escape sequence '\('
  GUE_PROPOSAL_REVIEWER_SYSTEM_prompt = """
c:\ChengJunyan1\Research\model_discovery\notebooks\..\model_discovery\agents\prompts\prompts.py:2325: SyntaxWarning: invalid escape sequence '\('
  GUE_DESIGNER_SYSTEM_prompt_part4 = """
c:\ChengJunyan1\Research\model_discovery\notebooks\..\model_discovery\agents\prompts\prompts.py:2631: SyntaxWarning: invalid escape sequence '\('
  GUE_IMPLEMENTATION_REVIEWER_SYSTEM_prompt = """


## Parsers

In [ ]:
import json
import os,sys
import importlib

sys.path.append('..')

import model_discovery.agents.flow.gau_utils
importlib.reload(model_discovery.agents.flow.gau_utils)
from model_discovery.agents.flow.gau_utils import check_and_reformat_gau_code
import model_discovery.agents.prompts.prompts as P
import model_discovery.model.composer
importlib.reload(model_discovery.model.composer)
from model_discovery.model.composer import GAUTree,GAUBase,GAUNode
import model_discovery.utils
importlib.reload(model_discovery.utils)
import model_discovery.utils as U



code='''
# gau.py   # DO NOT CHANGE OR REMOVE THE MAKK HERE, KEEP IT ALWAYS THE FIRST LINE #

import torch
import torch.nn as nn

from model_discovery.model.utils.modules import GAUBase # DO NOT CHANGE THIS IMPORT STATEMENT #
from model_discovery.model.utils.modules import GAUBase # DO NOT CHANGE THIS IMPORT STATEMENT #
from model_discovery.model.utils.modules import gau_test # DO NOT CHANGE THIS IMPORT STATEMENT #

import torch.nn.functional as F

# YOU CAN IMPORT MORE MODULES HERE #

# YOU CAN DEFINE MORE CLASSES OR FUNCTIONS HERE #

# Placeholder classes for future implementation
class MemoryAccessUnit(nn.Module):
    def __init__(self, embed_dim, memory_size, device=None, dtype=None):
        super().__init__(embed_dim)

    def _forward(self, X, **Z):
        return X, {}

class DownsamplingUnit(nn.Module):
    def __init__(self, embed_dim, downsample_factor, device=None, dtype=None):
        super().__init__(embed_dim)

    def _forward(self, X, **Z):
        return X, {}


class XXX(GAUBase,object):
    def __init__(self, embed_dim: int, device=None, dtype=None, **kwargs):
        super().__init__(embed_dim)


class GAU(GAUBase): # DO NOT CHANGE THE NAME OF THIS CLASS
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all current intermediate variables}
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable
    """
    def __init__(self, embed_dim: int, device=None, dtype=None,**kwargs): # YOU CAN ADD MORE ARGUMENTS WITH OPTIONAL DEFAULT VALUES, BUT YOU HAVE TO HAVE embed_dim, device, dtype AS THE ARGUTMENTS #
        # argv: list of hyperparameters
        factory_kwargs = {"device": device, "dtype": dtype} # remember to pass it to all nn layers
        super().__init__(embed_dim) # DO NOT CHANGE THIS LINE #

        self.a1 = kwarg_all.get('a1', 1)
        self.a2: int = kwarg_all.get('a2', 2)

        # COMPLETING THE CODE HERE #
        self.token_scorer = TokenScoringGAU(embed_dim, **factory_kwargs)
        self.dual_path: GAUBase = DualPathGAU(embed_dim, **factory_kwargs)
        self.latent_attention: GAUBase = LatentAttentionGAU(embed_dim, **factory_kwargs)

        self.stages: nn.ModuleList = nn.ModuleList([CompressionStageGAU(embed_dim, device=
            device, dtype=dtype, **kwargs) for _ in range(2)])

        self.modelx=nn.ModuleList([
            nn.Conv2d(1,20,5),
            nn.ReLU(),
            LatentAttentionGAU(),
            nn.Conv2d(20,64,5),
            nn.ReLU(),
        ])
            

        self.model1 = (
          nn.Conv2d(1,20,5),
          nn.ReLU(),
          LatentAttentionGAU(),
          nn.Conv2d(20,64,5),
          nn.ReLU(),
          nn.Sequential(OrderedDict([
                ('conv1', nn.Conv2d(1, 20, 5)),
                ('relu1', nn.ReLU()),
                ('gau2', DualPathGAU()),
                ('conv2', nn.Conv2d(20, 64, 5)),
                ('relu2', nn.ReLU())
            ]))
        )

        self.model = nn.Sequential(
          nn.Conv2d(1,20,5),
          nn.ReLU(),
          LatentAttentionGAU(),
          nn.Conv2d(20,64,5),
          nn.ReLU(),
          nn.Sequential(OrderedDict([
                ('conv1', nn.Conv2d(1, 20, 5)),
                ('relu1', nn.ReLU()),
                ('gau2', DualPathGAU()),
                ('conv2', nn.Conv2d(20, 64, 5)),
                ('relu2', nn.ReLU())
            ]))
        )

        self.activations = nn.ModuleDict({
            'relu': nn.ReLU(),
            'lrelu': nn.LeakyReLU(),
            'gau3': CompressionStageGAU()
        })

        self.model2 = nn.Sequential(OrderedDict([
          ('conv1', nn.Conv2d(1,20,5)),
          ('relu1', nn.ReLU()),
          ('gau2', DualPathGAU()),
          ('conv2', nn.Conv2d(20,64,5)),
          ('relu2', nn.ReLU())
        ]))

    # YOU CAN ADD MORE FUNCTIONS HERE #


    def init_something(self):
        self.modelx[3]=LatentAttentionGAU()
        self.modelx.append(LatentAttentionGAU())
        self.modelx.insert(0,LatentAttentionGAU())
        self.activations['gau1']=LatentAttentionGAU()
        self.model2.add_module('gau4',LatentAttentionGAU())

        X=self.dual_path(X)
    
    def _forward(self, X, **Z): 

        # THE CODE HERE MUST BE COMPLETED #
        # Step 1: Score tokens
        X, Z = self.token_scorer(X, **Z)
        # Step 2: Route through dual paths
        # Step 3: Apply latent attention
        Y, Z = self.latent_attention(X, A=A, B=B, **Z)
        for stage in self.stages:
            X = stage(X, **Z)
        
        X, Z = self.dual_path(X, **Z)
        X=self.activations['gau3'](X)

        return Y, Z

@gau_test
def test1(a=1,device=None):
    print('test1')

    
@gau_test
def test2(device=None):
    print('test2')


CHILDREN_DECLARATIONS = [
    UnitDecl(
        unitname='TokenScoringGAU', 
        requirements='Token scoring GAU',
        inputs=[],
        outputs=[],
    ),
    UnitDecl(
        unitname='DualPathGAU',
        requirements='Dual path GAU',
        inputs=[],
        outputs=[],
    ),
    UnitDecl(
        unitname='LatentAttentionGAU',
        requirements='Latent attention GAU',
        inputs=[],
        outputs=[],
    ),
    UnitDecl(
        unitname='CompressionStageGAU',
        requirements='Compression stage GAU',
        inputs=[],
        outputs=[],
    ),
]


'''

In [ ]:
code='''
# gau.py   # DO NOT CHANGE OR REMOVE THE MAKK HERE, KEEP IT ALWAYS THE FIRST LINE #

import torch
import torch.nn as nn
import torch.nn.functional as F

from model_discovery.model.utils.modules import GAUBase, gau_test # DO NOT CHANGE THIS IMPORT STATEMENT #


# YOU CAN IMPORT MORE MODULES HERE #

# YOU CAN DEFINE MORE CLASSES OR FUNCTIONS HERE #


class MHA(GAUBase):
    """
    MHA combines hierarchical gating mechanisms from HGRN2 with state space modeling (SSM) to improve long-range dependency capture and computational efficiency.

    This class replaces the traditional multi-head attention mechanism with a more efficient approach that maintains linear complexity with respect to sequence length.

    Args:
        embed_dim (int): The embedding dimension of the input sequence.
        block_loc (tuple): The location of the block within the network.
        kwarg_all (dict): Dictionary of additional keyword arguments for initialization.
        num_heads (int): The number of attention heads.
        expand_ratio (int): The ratio for expansion layers.
        d_state (int): The dimension of the state in the state space model.

    Returns:
        Y (Tensor): The output sequence with the same shape as the input.

    Example:
        >>> mha = MHA(embed_dim=512, block_loc=(0, 0), kwarg_all={}, num_heads=8, expand_ratio=4, d_state=64)
        >>> X = torch.randn(2, 10, 512)
        >>> Y, Z = mha(X)

    Note:
        This implementation maintains interpretability through the `get_attention_map` method, which provides a way to visualize effective attention patterns.
    """
    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict, num_heads: int, expand_ratio: int, d_state: int, device=None, dtype=None, **kwargs):
        self.factory_kwargs = {"device": device, "dtype": dtype} # DO NOT CHANGE THIS LINE, REMEMBER TO PASS IT #
        super().__init__(embed_dim, block_loc, kwarg_all) # DO NOT CHANGE THIS LINE #

        self.num_heads = num_heads
        self.expand_ratio = expand_ratio
        self.d_state = d_state
        self.head_dim = embed_dim // num_heads

        # HGRN2-inspired projections
        self.q_proj = nn.Linear(embed_dim, embed_dim, bias=True)
        self.k_proj = nn.Linear(embed_dim, embed_dim, bias=True)
        self.v_proj = nn.Linear(embed_dim, embed_dim, bias=True)

        # SSM components
        self.ssm_A = nn.Parameter(torch.randn(num_heads, d_state, d_state))
        self.ssm_B = nn.Parameter(torch.randn(num_heads, d_state, 1))
        self.ssm_C = nn.Parameter(torch.randn(num_heads, 1, d_state))

        # Expansion layers
        self.expand = nn.Linear(embed_dim, embed_dim * expand_ratio, bias=True)
        self.contract = nn.Linear(embed_dim * expand_ratio, embed_dim, bias=True)

        # Output projection
        self.out_proj = nn.Linear(embed_dim, embed_dim, bias=True)

        # Layernorm for stability
        self.norm = nn.LayerNorm(embed_dim)

        self._init_weights()

    def _init_weights(self):
        # Initialize SSM components
        nn.init.orthogonal_(self.ssm_A)
        nn.init.normal_(self.ssm_B, mean=0.0, std=0.02)
        nn.init.normal_(self.ssm_C, mean=0.0, std=0.02)

        # Initialize other weights
        nn.init.xavier_uniform_(self.q_proj.weight)
        nn.init.xavier_uniform_(self.k_proj.weight)
        nn.init.xavier_uniform_(self.v_proj.weight)
        nn.init.xavier_uniform_(self.expand.weight)
        nn.init.xavier_uniform_(self.contract.weight)
        nn.init.xavier_uniform_(self.out_proj.weight)

    def _forward(self, X, **Z):
        B, L, D = X.shape
        H = self.num_heads

        # HGRN2-inspired projections
        Q = self.q_proj(X).view(B, L, H, -1)
        K = self.k_proj(X).view(B, L, H, -1)
        V = self.v_proj(X).view(B, L, H, -1)

        # Expansion
        X_exp = self.expand(X).view(B, L, H, -1)

        # SSM state update
        state = torch.zeros(B, H, self.d_state, 1, device=X.device)
        outputs = []

        for t in range(L):
            # Update state
            state = torch.matmul(self.ssm_A, state) + self.ssm_B * K[:, t, :, :, None]

            # Compute output
            output = torch.matmul(self.ssm_C, state).squeeze(-1)
            output = output * Q[:, t, :, :]
            output = torch.sum(output * V[:, t, :, :], dim=-1)

            # Apply expansion
            output = output * X_exp[:, t, :, :]

            outputs.append(output)

        output = torch.stack(outputs, dim=1)
        output = output.view(B, L, -1)

        # Contraction
        output = self.contract(output)

        # Output projection and residual connection
        output = self.out_proj(output)
        output = self.norm(X + output)

        return output, Z

    def get_attention_map(self, X):
        B, L, D = X.shape
        H = self.num_heads

        Q = self.q_proj(X).view(B, L, H, -1)
        K = self.k_proj(X).view(B, L, H, -1)

        attention_map = torch.zeros(B, H, L, L, device=X.device)
        state = torch.zeros(B, H, self.d_state, 1, device=X.device)

        for t in range(L):
            state = torch.matmul(self.ssm_A, state) + self.ssm_B * K[:, t, :, :, None]
            output = torch.matmul(self.ssm_C, state).squeeze(-1)
            attention_map[:, :, t, :t+1] = torch.sum(output * Q[:, :t+1, :, :], dim=-1).transpose(1, 2)

        return attention_map


# WRITE YOUR UNIT TEST FUNCTIONS HERE #

@gau_test # DO NOT CHANGE THIS DECORATOR, OTHERWISE IT WON'T BE RECOGNIZED AS A UNIT TEST #
def test_mha(device=None, dtype=None)->None: # RENAME THIS FUNCTION, DO NOT CHANGE THE ARGUMENTS, IT SHOULD ALSO NOT RETURN ANYTHING #
    # AN AVAILABLE DEVICE AND DTYPE ARE PASSED AS ARGUMENTS, USE THEM TO INITIALIZE YOUR GAU AND MOCK INPUTS #
    embed_dim = 128
    block_loc = (0, 0)
    kwarg_all = {}
    num_heads = 8
    expand_ratio = 4
    d_state = 64
    mha = MHA(embed_dim, block_loc, kwarg_all, num_heads, expand_ratio, d_state, device=device, dtype=dtype)
    x = torch.randn(1, 100, 128).to(device=device, dtype=dtype)
    Z = {}
    y, Z_ = mha(x, **Z)
    assert y.shape == (1, 100, 128)

    # WRITE ASSERTIONS TO PERFORM THE TEST, USE PRINT TO DEBUG #

    print("Test passed!")

    # raise NotImplementedError # YOU MUST IMPLEMENT THIS FUNCTION #
'''
children=['ProjectionGAU','SimilarityComputationGAU','NormalizationGAU']

In [ ]:
unit_name = "MHA"  # Provide the unit_name to rename GAU class
reformatted_code, new_args, gau_tests, errors, warnings, fetal_errors,docstring,children_decl = check_and_reformat_gau_code(code, unit_name)
print("Reformatted Code:\n" + reformatted_code)
print("Errors:\n", '\n\t'.join(errors+fetal_errors))
print("Warnings:\n", '\n\t'.join(warnings))
print("New Arguments:\n", new_args)
print("Docstring:\n", docstring)
print("Children Declarations:\n", children_decl)



ckpt_dir = os.environ['CKPT_DIR']
lib_dir = U.pjoin(ckpt_dir, 'test_composer', 'lib')
test_tree = GAUTree('TestTree', None, None, None,None,lib_dir)
spec=P.UnitSpec(
   unitname = unit_name,
   document = 'Test',
   inputs = [],
   outputs = [],
)
# test_tree.add_unit(
#     spec, reformatted_code, new_args, None, None, None, children, gau_tests, None
# )


In [ ]:
x=[{'role': 'system', 'content': '\nYou are a professional AI researcher focusing on discovering the best\nautoregressive language model block. Your goal is to design a novel block\nfollowing the Generalized Autoregressive Block (GAB) structure defined in the\nfollowing base class:\n\n```python class GABBase(nn.Module):\n """ Base class for Generalized Autoregressive Block """\n def __init__(self,embed_dim: int, block_loc: tuple):\n super().__init__()\n self.embed_dim = embed_dim\n self.block_loc = block_loc # location of a block within the network, (layer_idx, n_block)\n\n def _forward(self, X, **Z): \n raise NotImplementedError\n \n # YOU ARE NOT ALLOW TO OVERRIDE THIS METHOD #\n def forward(self, X, **Z): # kwargs not parsable by torchscript but more flexible\n """Forward pass of the model"""\n assert len(X.shape) == 3, f"Input shape must be (batch, seqlen, embed_dim), got {len(X.shape)}D instead"\n assert X.shape[-1] == self.embed_dim, f"Input shape must be (batch, seqlen, embed_dim), got {X.shape} instead"\n Y = self._forward(X, **Z)\n if isinstance(Y, tuple):\n Y, Z_ = Y\n else:\n Z_ = {}\n assert Y.shape == X.shape, f"GAB Output shape must be the same as input shape of {X.shape}, got {Y.shape} instead"\n assert isinstance(Z, dict), "Intermediate variables must be stored in a dict"\n Z.update(Z_) # the new intermediate variables are updated to the current Z\n return Y, Z\n ```\n\nThe GAB will be used to construct a Generalized Autoregressive Model (GAM)\ndefined as follows:\n\n```python from transformers.modeling_outputs import CausalLMOutput\nfrom transformers import PreTrainedModel, PretrainedConfig\nfrom dataclasses import dataclass\n\nimport torch\nfrom torch import nn\n\nfrom gab import GAB, gab_config\n\n\n@dataclass\nclass GAMConfig(PretrainedConfig):\n \'\'\'Configurations for Generalized Autoregressive Models.\'\'\'\n\n d_model: int\n n_block: int\n batch_tokens: int \n vocab_size: int = None\n\n\nclass GAM(nn.Module):\n \'\'\' Generalized Autoregressive Models\n Input: X: (batch, seqlen, embed_dim)\n Output: Y: (batch, seqlen, embed_dim)\n \'\'\'\n def __init__(\n self,\n d_model: int,\n n_block: int,\n vocab_size: int = 50277,\n norm_epsilon: float = 1e-5,\n device = None,\n dtype = None,\n ) -> None:\n self.factory_kwargs = {"device": device, "dtype": dtype}\n super().__init__()\n self.d_model = d_model\n self.embedding = nn.Embedding(vocab_size, d_model, **self.factory_kwargs)\n\n block_config = gab_config()\n self.blocks = nn.ModuleList(\n [\n GAB(\n embed_dim=d_model, \n block_loc=(layer_idx,n_block),\n device=device, \n dtype=dtype, \n **block_config\n )\n for layer_idx in range(n_block)\n ]\n )\n self.norm_out = nn.LayerNorm(\n d_model, eps=norm_epsilon, **self.factory_kwargs\n )\n\n def forward(self, input_ids):\n hidden_states = self.embedding(input_ids)\n intermediate_vars = {}\n for block in self.blocks:\n hidden_states, intermediate_vars = block(\n hidden_states,\n **intermediate_vars\n )\n hidden_states = self.norm_out(hidden_states)\n return hidden_states\n\n\nclass GLMHeadModel(PreTrainedModel):\n \'\'\' Generalized Autoregressive Models with LM Head \'\'\'\n config_class = GAMConfig\n\n def __init__(\n self,\n config: GAMConfig,\n device=None,\n dtype=None,\n ) -> None:\n super().__init__(config)\n factory_kwargs = {"device": device, "dtype": dtype}\n self.backbone = GAM(\n d_model=config.d_model,\n n_block=config.n_block,\n vocab_size=config.vocab_size,\n **factory_kwargs,\n )\n self.lm_head = nn.Linear(config.d_model, config.vocab_size, bias=False, **factory_kwargs)\n\n def forward(self, input_ids, **gab_kwargs):\n hidden_states = self.backbone(input_ids, **gab_kwargs)\n lm_logits = self.lm_head(hidden_states)\n return CausalLMOutput(logits=lm_logits)\n ```\n\nThe produced language model will be pretrained with the corpus and then be\napplied for downstream tasks. The new model is expected to have a low\nperplexity, high accuracy, robustness, efficiency, and most importantly, good\nscalability. \n\nSince the autoregressive model design is complicated, so we will break it down\ninto smaller parts. We represent a block as multiple nested units, the\nGeneralized Autoregressive Unit (GAU). Each GAU accepts a sequence of embeddings\nX and a dictionary of intermediate variables Z as input, and outputs a sequence\nof embeddings Y and a dictionary of new or updated intermediate variables Z_. Z_\nis optional, when it is provided, it will be used to update Z for the next unit\nby Z.update(Z_). A GAU is defined in the following base class:\n\n```python class GAUBase(nn.Module): \n """ \n Instead of directly giving the full implementation of a GAB block, the agent need to \n design a series of nested GAB units and construct the full GAB block as a pipeline of these units.\n\n GAB is fractal, like GAB itself, each GAB unit accepts X and Z as input and returns Y and Z as output.\n """ \n def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict):\n super().__init__()\n self.embed_dim = embed_dim\n self.block_loc = block_loc # location of a block within the network, (layer_idx, n_block)\n self.kwarg_all = kwarg_all # dict of all kwargs, use it to initialize the children units\n\n def _forward(self, X, **Z): # you can define the arguments instead of using **Z\n raise NotImplementedError\n \n def forward(self, X, **Z):\n assert len(X.shape) == 3 and X.shape[-1] == self.embed_dim, f"Input X must be a sequence of shape (batch, seqlen, embed_dim), all other kinds of variables should be passed by Z. Got {X.shape} instead."\n _params = inspect.signature(self._forward).parameters\n X=X.to(**self.factory_kwargs)\n _Z = {k: v for k, v in Z.items() if k in _params}\n Y = self._forward(X, **_Z)\n if isinstance(Y, tuple):\n Y, Z_ = Y\n else:\n Z_ = {}\n assert Y.shape == X.shape, f"GAU output Y must be a sequence with the same shape as input of {X.shape}, got {Y.shape} instead, all other kinds of variables should be passed by Z."\n assert isinstance(Z_, dict), "Intermediate variables must be stored in a dict"\n Z.update(Z_) # the new intermediate variables are updated to the current Z\n return Y, Z\n ```\n\nYou will design a GAU by completing the blanks marked in this template, which\nincludes the initialization where you can define your custom arguments with\noptional default values, the forward function where you can define convenient\nfunctions or classes in the GAB class such as caches, notice that you are only\nallowed to have only one GAU which inherited from the GAUBase class in the file:\n \n```python # gau.py # DO NOT CHANGE OR REMOVE THE MAKK HERE, KEEP IT ALWAYS THE FIRST LINE #\n\nimport torch\nimport torch.nn as nn\n\nfrom model_discovery.model.utils.modules import GAUBase, gau_test # DO NOT CHANGE THIS IMPORT STATEMENT #\n\n\n# YOU CAN IMPORT MORE MODULES HERE #\n\n# YOU CAN DEFINE MORE CLASSES OR FUNCTIONS HERE #\n\n\nclass UnitName(GAUBase):\n """Generalized Autoregressive Block Unit\n Input: X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} \n Output: Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}\n Constraints: Causal, differentiable, parameter number, complexity, parallelizable...\n\n embed_dim: The dimension of the input embeddings\n block_loc: The location of the block to be composed within the network, (layer_idx, n_block), e.g. (0, 6) for the first block in a network with 6 blocks in total\n kwarg_all: A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units\n """\n def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict, device=None, dtype=None,**kwargs): # YOU CAN ADD MORE ARGUMENTS WITH OPTIONAL DEFAULT VALUES, BUT KEEP THE ORIGINAL ONES #\n self.factory_kwargs = {"device": device, "dtype": dtype} # DO NOT CHANGE THIS LINE, REMEMBER TO PASS IT #\n super().__init__(embed_dim, block_loc, kwarg_all) # DO NOT CHANGE THIS LINE #\n \n # COMPLETING THE CODE HERE #\n\n raise NotImplementedError\n\n\n # YOU CAN ADD MORE FUNCTIONS HERE #\n\n\n def _forward(self, X, **Z): \n \n # THE CODE HERE MUST BE COMPLETED #\n\n raise NotImplementedError\n\n\n# WRITE YOUR UNIT TEST FUNCTIONS HERE #\n\n@gau_test # DO NOT CHANGE THIS DECORATOR, OTHERWISE IT WON\'T BE RECOGNIZED AS A UNIT TEST #\ndef unit_test_name(device=None, dtype=None)->None: # RENAME THIS FUNCTION, DO NOT CHANGE THE ARGUMENTS, IT SHOULD ALSO NOT RETURN ANYTHING #\n # AN AVAILABLE DEVICE AND DTYPE ARE PASSED AS ARGUMENTS, USE THEM TO INITIALIZE YOUR GAU AND MOCK INPUTS #\n\n # WRITE ASSERTIONS TO PERFORM THE TEST, USE PRINT TO DEBUG #\n \n raise NotImplementedError # YOU MUST IMPLEMENT THIS FUNCTION #\n ```\n\nIn a GAU, you can call other GAUs, as such, you can create a complicated GAB\nblock by nesting multiple GAUs. However, each GAU should be not too complex, if\nyou want to create complex block, you should break it down into smaller GAUs and\nnest them. As such, you should design a GAB block in a top-down manner. \n\nInstead of starting from scratch, you will start from an existing design and\nimprove it. You will be provided with the full information of the design,\nincluding the proposal, the tree structure, the implementations of the GAUs. You\nare only allowed to modify *one GAU* from the existing design. \n\nYou need to select one GAU to modify, you can define new children GAUs, however\nyou need to guarantee that your modification wont affect the correctness of the\noverall design. \n\nYou will start by writing down an overal proposal for the design you want to\nhave, the proposal decides a direction, phylosophy and the plan of the design,\nand the analysis of the problem and how you gonna solve it by modifying one GAU\nfrom the existing design. You will be provided with one or multiple references\nto consider that may inspire you if there are references provided.\n\nYour response should include: \n\n1. The proposal, it should include but not restrict to the following parts: a. A\n title with the name of the design in the level 1 header format. You shuld\n have only one level 1 header in your response which is the name of the\n design.\n\n b. Your motivation of the design. What problem you want to solve based on the\n insights or observations you have about the autoregressive models today,\n and any inspirations you may have from the references. \n\n c. The analysis of the problem.\n\n d. The core idea and phylosophy behind of your design that may solve the\n problem you proposed. \n\n e. The plan of the design. You should include subsections of that describe\n the details of each part of the design with the justifications. The\n selection of the GAU to modify and the reasoning of the selection. \n\n f. A conclution of the proposal. \n\n g. Optional, the references you used in your proposal, should be in the right\n format.\n2. The name of the variant of the model you are going to design.\n3. The selection of the GAU to modify.\n\nThe proposal will be reviewed and you will be asked to modify it if it is not\npassed. You can start to implement the design after the proposal is passed. \n\nThe proposal should be as detailed as possible, DO NOT WORRY IF THE PROPOSAL IS\nTOO LONG, BUT ALSO DO NOT FILL IN BY REDUNDANT WORDS, USE PRECISE AND CONCRETE\nLANGUAGE, the proposal will be the guideline for the entire design process so it\nshould be clear and detailed. \n'}]
print(x[0]['content'])

In [ ]:
code=test_tree.compose_unit('XAU')
code=U.add_line_num(code)
print(code)

In [ ]:
code='''
import torch
import torch.nn as nn
from model_discovery.model.utils.modules import GAUBase, gau_test


class DilatedReformerRootGAU(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block to be composed within the network, (layer_idx, n_block), e.g. (0, 6) for the first block in a network with 6 blocks in total
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.dilated_attention = DilatedAttentionGAU(embed_dim=embed_dim,
            block_loc=block_loc, kwarg_all=kwarg_all, **self.factory_kwargs,
            **kwarg_all)
        self.reversible_residual = ReversibleResidualGAU(embed_dim=
            embed_dim, block_loc=block_loc, kwarg_all=kwarg_all, **
            self.factory_kwargs, **kwarg_all)
        self.lsh_attention = LSHAttentionGAU(embed_dim=embed_dim, block_loc
            =block_loc, kwarg_all=kwarg_all, **self.factory_kwargs, **kwarg_all
            )

    def _forward(self, X, **Z):
        X, Z = self.dilated_attention(X, **Z)
        X, Z = self.reversible_residual(X, **Z)
        X, Z = self.lsh_attention(X, **Z)
        return X, Z


class LSHAttentionGAU(GAUBase): 
    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict, device=None, dtype=None, **kwargs): 
        self.factory_kwargs = {"device": device, "dtype": dtype} 
        super().__init__(embed_dim, block_loc, kwarg_all)

    def _forward(self, X, **Z): 
        return X


class DilatedAttentionGAU(GAUBase): 
    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict, device=None, dtype=None, **kwargs): 
        self.factory_kwargs = {"device": device, "dtype": dtype} 
        super().__init__(embed_dim, block_loc, kwarg_all)

    def _forward(self, X, **Z): 
        return X


class ReversibleResidualGAU(GAUBase): 
    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict, device=None, dtype=None, **kwargs): 
        self.factory_kwargs = {"device": device, "dtype": dtype} 
        super().__init__(embed_dim, block_loc, kwarg_all)

    def _forward(self, X, **Z): 
        return X


@gau_test
def test_DilatedReformerRootGAU_unit_test_dilated_reformer_root_gau(device=
    None, dtype=None) ->None:
    batch_size = 2
    seq_len = 10
    embed_dim = 16
    X = torch.randn(batch_size, seq_len, embed_dim, device=device, dtype=dtype)
    Z = {}
    gau = DilatedReformerRootGAU(embed_dim=embed_dim, block_loc=(0, 6), kwarg_all={}, device=
        device, dtype=dtype)
    Y, Z_ = gau(X, **Z)
    assert Y.shape == X.shape, 'Output shape should match input shape'
    assert isinstance(Z_, dict), 'Z_ should be a dictionary'
    print('Input X:', X)
    print('Output Y:', Y)
    print('Intermediate variables Z_:', Z_)


def run_DilatedReformerRootGAU_tests():
	try:
		test_DilatedReformerRootGAU_unit_test_dilated_reformer_root_gau()
	except Exception as e:
		print("Error in running unit_test_dilated_reformer_root_gau:", e)


if __name__ == "__main__":
	run_DilatedReformerRootGAU_tests()
'''

import io
from contextlib import redirect_stdout

output=io.StringIO()
with redirect_stdout(output):
    exec(code)
captured = str(output.getvalue())

## Debug area

In [ ]:
import torch
import torch.nn as nn
from model_discovery.model.utils.modules import GABBase


class GAB(GABBase):

    def __init__(self, embed_dim: int, block_loc: tuple, device=None, dtype
        =None, **kwargs):
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc)
        self.root = ResonanceLOCOSTGAMRoot(embed_dim=embed_dim, block_loc=
            block_loc, kwarg_all=kwargs, **factory_kwargs, **kwargs)

    def _forward(self, X, **Z):
        X, Z = self.root(X, **Z)
        return X, Z


from model_discovery.model.utils.modules import GAUBase, gau_test


class ResonanceLOCOSTGAMRoot(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block to be composed within the network, (layer_idx, n_block), e.g. (0, 6) for the first block in a network with 6 blocks in total
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.resonance_rope = ResonanceRoPEUnit(embed_dim=embed_dim,
            block_loc=block_loc, kwarg_all=kwarg_all, **self.factory_kwargs,
            **kwarg_all)
        self.state_space = StateSpaceUnit(embed_dim=embed_dim, block_loc=
            block_loc, kwarg_all=kwarg_all, **self.factory_kwargs, **kwarg_all)
        self.inter_block_comm = InterBlockCommUnit(embed_dim=embed_dim,
            block_loc=block_loc, kwarg_all=kwarg_all, **self.factory_kwargs,
            **kwarg_all)

    def _forward(self, X, **Z):
        Y, Z_ = self.resonance_rope(X, **Z)
        Y, Z_ = self.state_space(Y, **Z_)
        Y, Z_ = self.inter_block_comm(Y, **Z_)
        return Y, Z_


import math


class ResonanceRoPEUnit(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block to be composed within the network, (layer_idx, n_block), e.g. (0, 6) for the first block in a network with 6 blocks in total
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.freqs = self._init_frequencies(embed_dim)

    def _init_frequencies(self, embed_dim):
        half_dim = embed_dim // 2
        freqs = torch.exp(-torch.arange(half_dim, dtype=torch.float32,
            device=self.factory_kwargs['device']) * (math.log(10000.0) / (
            half_dim - 1)))
        return freqs

    def _apply_rope(self, X):
        batch_size, seqlen, _ = X.size()
        position_ids = torch.arange(seqlen, dtype=X.dtype, device=X.device)
        sinusoid_inp = torch.einsum('i,j->ij', position_ids, self.freqs)
        sin, cos = sinusoid_inp.sin(), sinusoid_inp.cos()
        X1, X2 = X[..., ::2], X[..., 1::2]
        Y1 = X1 * cos - X2 * sin
        Y2 = X1 * sin + X2 * cos
        return torch.stack([Y1, Y2], dim=-1).flatten(-2)

    def _forward(self, X, **Z):
        Y = self._apply_rope(X)
        Z_ = Z
        return Y, Z_


class StateSpaceUnit(GAUBase):

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)

    def _forward(self, X, **Z):
        return X


class InterBlockCommUnit(GAUBase):

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)

    def _forward(self, X, **Z):
        return X
    


gab_config = {}


# Check if to device still work!

gab = GAB(128, (0, 6))

In [ ]:
def get_module_device(model):
    """
    Returns the device of the first parameter found in the given nn.Module.
    If the module has no parameters, returns None.
    """
    return next(model.parameters()).device

get_module_device(gab.root)


In [ ]:
next(gab.parameters())

## Tree

In [ ]:
children={
    'unit1': 'MockUnit1',
    'unit2': 'MockUnit2',
}
called=['unit1', 'unit2', 'unit1']
test_tree.units['DualPathGAU'] = GAUNode('DualPathGAU', None, None, None, called, None, None, children, None)
children={
    'unit1': 'MockUnit3',
    'unit2': 'MockUnit4',
}
called=['unit2', 'unit2']
test_tree.units['MockUnit1'] = GAUNode('MockUnit1', None, None, None, called, None, None, children, None)

In [ ]:
test_tree.units

In [ ]:
class A:
    def __init__(self,r,d,e,**Z):
        print('Args in A:',d,e)
        print('Kwargs in A:',Z)

class B(A):
    def __init__(self,_Z,c=1,d=2,e=3,**Z):
        args2={'d':d,'e':e}
        super().__init__(c,**_Z,**args2)
        print('Kwargs in B:',Z)

cfg={
    'a':1,
    'b':2,
    'c':3,
}

B(cfg,**cfg)

In [ ]:
def A(X,Z,**kwargs):
    print(X)
    print(Z)
    print(kwargs)

def B(X,Z,a,**kwargs):
    A(X,Z,**Z)

Z={'a':3}
B(1,Z,**Z)

## Test


In [ ]:
import os,sys
sys.path.append('..')
import torch
import torch.nn as nn
from model_discovery.model.utils.modules import GABBase


class GAB(GABBase):
    '''
    GAU Tree Map of Butterfly-AFT Generalized Autoregressive Block (BAGAB):
    ButterflyAFTGAU (Rating: 3.5/5)
        |- AFTMechanismGAU (Rating: 3.5/5)
            |- PositionBiasGAU (Rating: 3.5/5)
            |- ElementWiseOperationGAU (Rating: 3.5/5)
        |- ButterflyTransformGAU (Rating: 3.5/5)
            |- ButterflyLayerGAU (Rating: 3.5/5)
                |- ButterflyStageGAU (Rating: 3.5/5)
                    |- ButterflyParameterGAU (Unimplemented)
            |- ButterflyInitializationGAU (Rating: 3.5/5)
                |- ButterflyStageGAU (Rating: 3.5/5)
                    |- ButterflyParameterGAU (Unimplemented)
                |- ButterflyParameterGAU (Unimplemented)

    Implemented Units: AFTMechanismGAU, ButterflyLayerGAU, PositionBiasGAU, ButterflyTransformGAU, ElementWiseOperationGAU, ButterflyAFTGAU, ButterflyInitializationGAU, ButterflyStageGAU
    Unimplemented Units: AFTMechanismGAU, ButterflyLayerGAU, PositionBiasGAU, ButterflyTransformGAU, ButterflyParameterGAU, ElementWiseOperationGAU, ButterflyInitializationGAU, ButterflyStageGAU
    '''

    def __init__(self, embed_dim: int, block_loc: tuple, device=None, dtype
        =None, **kwargs):
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc)
        self.root = ButterflyAFTGAU(embed_dim=embed_dim, block_loc=
            block_loc, kwarg_all=kwargs, **factory_kwargs, **kwargs)

    def _forward(self, X, **Z):
        X, Z = self.root(X, **Z)
        return X, Z


from model_discovery.model.utils.modules import GAUBase


class ButterflyAFTGAU(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block within the network, (layer_idx, n_block)
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.aft_mechanism = AFTMechanismGAU(embed_dim=embed_dim, block_loc
            =block_loc, kwarg_all=kwarg_all, **self.factory_kwargs, **kwarg_all
            )
        self.butterfly_transform = ButterflyTransformGAU(embed_dim=
            embed_dim, block_loc=block_loc, kwarg_all=kwarg_all, **self.
            factory_kwargs, **kwarg_all)

    def _forward(self, X, **Z):
        aft_output, Z = self.aft_mechanism(X, **Z)
        butterfly_output, Z = self.butterfly_transform(aft_output, **Z)
        return butterfly_output, Z


class ButterflyTransformGAU(GAUBase):

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.butterfly_layer = ButterflyLayerGAU(embed_dim=embed_dim,
            block_loc=block_loc, kwarg_all=kwarg_all, **self.factory_kwargs,
            **kwarg_all)
        self.butterfly_init = ButterflyInitializationGAU(embed_dim=
            embed_dim, block_loc=block_loc, kwarg_all=kwarg_all, **self.
            factory_kwargs, **kwarg_all)

    def _forward(self, X, **Z):
        X, Z = self.butterfly_init(X, **Z)
        Y, Z = self.butterfly_layer(X, **Z)
        return Y, Z


class ButterflyLayerGAU(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block within the network, (layer_idx, n_block)
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.stages = nn.ModuleList([ButterflyStageGAU(embed_dim=embed_dim,
            block_loc=block_loc, kwarg_all=kwarg_all, **self.factory_kwargs,
            **kwarg_all) for _ in range(kwarg_all.get('num_stages', 4))])

    def _forward(self, X, **Z):
        for stage in self.stages:
            X, Z = stage(X, **Z)
        return X, Z

class ButterflyMatrixGAU(GAUBase):

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)

    def _forward(self, X, **Z):
        return X


import torch.nn.functional as F


class ButterflyInitializationGAU(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block within the network, (layer_idx, n_block)
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.butterfly_size = kwarg_all.get('butterfly_size', embed_dim)
        self.butterfly_matrices = nn.ParameterList([nn.Parameter(torch.
            randn(self.butterfly_size, self.butterfly_size, **self.
            factory_kwargs)) for _ in range(int(torch.log2(torch.tensor(
            self.butterfly_size))))])
        self.butterfly_stage = ButterflyStageGAU(embed_dim=embed_dim,
            block_loc=block_loc, kwarg_all=kwarg_all, **self.factory_kwargs,
            **kwarg_all)
        self.butterfly_parameter = ButterflyParameterGAU(embed_dim=
            embed_dim, block_loc=block_loc, kwarg_all=kwarg_all, **self.
            factory_kwargs, **kwarg_all)

    def _forward(self, X, **Z):
        for matrix in self.butterfly_matrices:
            X = F.linear(X, matrix)
        return X, Z


class ButterflyParameterGAU(GAUBase):
    """
    Generalized Autoregressive Block Unit for learning parameters of Butterfly Factorization.
    Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
    Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
    Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

    embed_dim:    The dimension of the input embeddings
    block_loc:    The location of the block within the network, (layer_idx, n_block)
    kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.param_size = kwarg_all.get('param_size', 128)
        self.butterfly_weights = nn.Parameter(torch.randn(self.param_size,
            embed_dim, **self.factory_kwargs))
        nn.init.xavier_uniform_(self.butterfly_weights)

    def _forward(self, X, **Z):
        transformed_X = torch.matmul(X, self.butterfly_weights)
        Z_ = {'butterfly_weights': self.butterfly_weights}
        return transformed_X, Z_
        # transformed_X = F.linear(X, self.butterfly_weights.T)
        # return transformed_X, {}


class ButterflyComputationGAU(GAUBase):

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)

    def _forward(self, X, **Z):
        return X


class ButterflyStageGAU(GAUBase):

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.butterfly_params = ButterflyParameterGAU(embed_dim=embed_dim,
            block_loc=block_loc, kwarg_all=kwarg_all, **self.factory_kwargs,
            **kwarg_all)

    def _forward(self, X, **Z):
        transformed_X, Z = self.butterfly_params(X, **Z)
        assert transformed_X.shape == X.shape, f'Output shape {transformed_X.shape} does not match input shape {X.shape}'
        return transformed_X, Z

class AFTMechanismGAU(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block within the network, (layer_idx, n_block)
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.position_bias = PositionBiasGAU(embed_dim=embed_dim, block_loc
            =block_loc, kwarg_all=kwarg_all, **self.factory_kwargs, **kwarg_all
            )
        self.element_wise_operation = ElementWiseOperationGAU(embed_dim=
            embed_dim, block_loc=block_loc, kwarg_all=kwarg_all, **self.
            factory_kwargs, **kwarg_all)

    def _forward(self, X, **Z):
        X, Z = self.position_bias(X, **Z)
        Y, Z = self.element_wise_operation(X, **Z)
        return Y, Z


class ElementWiseOperationGAU(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block within the network, (layer_idx, n_block)
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.position_bias = nn.Parameter(torch.zeros(1, embed_dim, **self.
            factory_kwargs))

    def _forward(self, X, **Z):
        queries = Z.get('queries', X)
        keys = Z.get('keys', X)
        values = Z.get('values', X)
        elementwise_product = queries * keys
        elementwise_sum = elementwise_product + values + self.position_bias
        return elementwise_sum, {}


class PositionBiasGAU(GAUBase):
    """Position Bias Generalized Autoregressive Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block within the network, (layer_idx, n_block)
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.position_bias = nn.Parameter(torch.zeros(1, 1, embed_dim, **
            self.factory_kwargs))

    def _forward(self, X, **Z):
        Y = X + self.position_bias
        return Y, {}
    

class GABLM(nn.Module):
    def __init__(self, embed_dim, vocab_size=100, **kwargs):
        super().__init__()
        self.gab = GAB(embed_dim, (0,1), **kwargs)
        self.lm_head = nn.Linear(embed_dim, vocab_size, bias=False)
    
    def forward(self, X, **Z):
        X, Z = self.gab(X, **Z)
        return self.lm_head(X),X



gab_config = {}


In [ ]:
class SimpleGABLM(nn.Module):
    def __init__(self, embed_dim, vocab_size=100):
        super().__init__()
        self.butterfly_param = ButterflyParameterGAU(embed_dim, (0,1), {})
        self.lm_head = nn.Linear(embed_dim, vocab_size, bias=False)
    
    def forward(self, X, **Z):
        X, _ = self.butterfly_param(X, **Z)
        return self.lm_head(X), X

# Test the simplified model
simple_model = GABLM(128)
mock_input = torch.randn(1, 10, 128)
mock_y = torch.randint(0, 100, (1, 10))

optimizer = optim.Adam(simple_model.parameters())
criterion = nn.CrossEntropyLoss()

# Training loop
for _ in range(5):  # Run for a few iterations
    optimizer.zero_grad()
    y, X = simple_model(mock_input)
    loss = criterion(y.view(-1, 100), mock_y.view(-1))
    loss.backward()
    print(f"Loss: {loss.item()}")
    for name, param in simple_model.named_parameters():
        if param.grad is not None:
            print(f"Gradient for {name}: {param.grad.abs().mean()}")
        else:
            print(f"No gradient for {name}")
    optimizer.step()

In [ ]:
from torch import nn, optim 

model=GABLM(128, **gab_config)
vocab_size=100
DEFAULT_CONTEXT_LENGTH=10

mock_input=torch.randn(1, DEFAULT_CONTEXT_LENGTH, 128)
mock_y=torch.randint(0, 100, (1, 10))
mock_Z={}


criterion = nn.CrossEntropyLoss()
model.train()
optimizer = optim.Adam(model.parameters())

# Zero the parameter gradients
optimizer.zero_grad()
y,X=model(mock_input, **mock_Z)
loss=nn.CrossEntropyLoss()(y.view(-1, 100), mock_y.view(-1))
loss.backward()

used_params = set()

# Hook to register which parameters are used during the forward pass
def hook_fn(module, input, output):
    for param in module.parameters():
        used_params.add(param)

# Register hooks for all submodules
hooks = []
for submodule in model.modules():
    hook = submodule.register_forward_hook(hook_fn)
    hooks.append(hook)

# Forward pass again to collect used parameters
model(mock_input)

# Remove all hooks
for hook in hooks:
    hook.remove()

all_gradients_present = True

# Check gradients
for name, param in model.named_parameters():
    if param.requires_grad:
        if param in used_params and param.grad is None:
            # Used parameter without gradient -> Error
            print(f"Error: Used parameter {name} requires gradients but has none.")
            all_gradients_present = False
        elif param not in used_params:
            # Unused parameter -> Warning
            print(f"Warning: Parameter {name} was not used in the forward pass.")

if all_gradients_present:
    print('Differentiability test passed')
else:
    print('Differentiability test failed due to missing gradients.')


## Test Claude

In [ ]:
TEST_INIT_DESIGN_INPUT='''
Below is the specification for the GAU you need to refine:
Specification: Unit Name: GatedMLP The DynamicGatedMLP is an enhanced version of the GatedMLP that introduces a dynamic gating mechanism capable of selecting between multiple activation functions. This design aims to improve the expressiveness and adaptability of the model by allowing it to dynamically choose the most suitable activation function for different inputs. The unit maintains the same input-output interface as the original GatedMLP, ensuring seamless integration with the existing GPT-2 architecture.


Inputs: X
Outputs: Y
Children list: []

Current Implementation: import torch import torch.nn as nn from model_discovery.model.utils.modules import GAUBase, gau_test import torch.nn.functional as F

class GatedMLP(GAUBase): """Generalized Autoregressive Block Unit Input: X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} Output: Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of new intermediate variables to update the current Z} Constraints: Causal, differentiable, parameter number, complexity, parallelizable...

embed_dim:    The dimension of the input embeddings
    block_loc:    The location of the block to be composed within the network, (layer_idx, n_block), e.g. (0, 6) for the first block in a network with 6 blocks in total
    kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
"""

def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
    device=None, dtype=None, hidden_features=None, out_features=None,
    bias=False, multiple_of=128, **kwargs):
    self.factory_kwargs = {'device': device, 'dtype': dtype}
    super().__init__(embed_dim, block_loc, kwarg_all)
    out_features = out_features if out_features is not None else embed_dim
    hidden_features = (hidden_features if hidden_features is not None else
        int(8 * embed_dim / 3))
    hidden_features = (hidden_features + multiple_of - 1
        ) // multiple_of * multiple_of
    self.fc1 = nn.Linear(embed_dim, 2 * hidden_features, bias=bias, **
        self.factory_kwargs)
    self.fc2 = nn.Linear(hidden_features, out_features, bias=bias, **
        self.factory_kwargs)
    self.activations = [F.relu, F.tanh, F.gelu]
    self.gate_weights = nn.Parameter(torch.randn(len(self.activations),
        hidden_features, **self.factory_kwargs))

def _forward(self, X, **Z):
    y = self.fc1(X)
    y, gate = y.chunk(2, dim=-1)
    gate_scores = F.softmax(self.gate_weights, dim=0)
    y = sum(gate_scores[i] * self.activations[i](gate) for i in range(
        len(self.activations)))
    y = self.fc2(y)
    return y, {}

Review: ### Highlights:

Successful Implementation Fix: The designer has successfully addressed the previous implementation error by initializing the activations attribute before its use, resolving the AttributeError.
Dynamic Gating Mechanism: The design continues to leverage a dynamic gating mechanism, which is a novel approach that enhances the model's adaptability and expressiveness by selecting between multiple activation functions.
Checker Validation: The updated implementation has passed all checker tests, including forward pass, causality, and differentiability, indicating that the GAU is functioning correctly within the model.
Concerns:
Efficiency Considerations: While the implementation is now functional, the potential computational overhead of evaluating multiple activation functions simultaneously remains a concern. This could impact efficiency, especially in larger models or datasets.
Justification of Activation Functions: The choice of activation functions (ReLU, Tanh, GELU) is reasonable, but further theoretical justification or empirical validation would strengthen the design.
Accuracy, Robustness, Efficiency, and Scalability:
Accuracy and Robustness: The dynamic selection of activation functions should enhance the model's ability to adapt to various input patterns, potentially improving accuracy and robustness.
Efficiency: The design could still benefit from optimizations to reduce computational overhead, ensuring it remains efficient in practice.
Scalability: The design appears scalable, as indicated by the successful checker tests, but empirical validation on larger datasets would be beneficial.
Novelty:
The design introduces a novel dynamic gating mechanism, which is a significant improvement over static activation functions.
Clarity and Completeness:
The design is now clear and complete, with the previous implementation error resolved.
Theoretical Soundness:
The theoretical foundation of using dynamic gating is sound, and the implementation aligns well with the proposal's objectives.
Implementation Feasibility:
The current implementation is feasible and has been validated by the checker, indicating it can be integrated into the broader model.
Suggestions:
Efficiency Optimization: Consider exploring methods to optimize the dynamic gating mechanism to reduce computational overhead, such as selectively applying it or using more efficient computation techniques.
Empirical Validation: Plan for empirical validation to confirm the theoretical benefits of the dynamic gating mechanism and the choice of activation functions in practice.
Documentation: Ensure thorough documentation of the design choices and their theoretical justifications to aid future refinements and empirical testing.
Rating: 4 out of 5 (Passing score >3)

Reviewer Suggestions: 1. Efficiency Optimization: Consider exploring methods to optimize the dynamic gating mechanism to reduce computational overhead, such as selectively applying it or using more efficient computation techniques. 2. Empirical Validation: Plan for empirical validation to confirm the theoretical benefits of the dynamic gating mechanism and the choice of activation functions in practice.

Documentation: Ensure thorough documentation of the design choices and their theoretical justifications to aid future refinements and empirical testing.
Refinement Process
If there is a review provided, you should start by reflecting on the feedback. Otherwise, leave reflection empty. The, proceed with the following:

New Analysis and Design: - Provide an updated detailed analysis based on the feedback, including your new design direction and justifications. - Include a high-level pseudocode that captures the core of the new design. You should also provide the updated document of the GAU that allows other people to understand the design and implementation without the need to read the code.

Implementation: - Provide the full updated implementation of the GAU, following the specified format and templates.

Children list: - Provide the list of the children GAUs that are declared in the current GAU. You can declare new children GAUs or preserve the existing ones. If you do not declare any new children GAUs, you should provide the original children GAUs.

Log of Changes: - Summarize the key changes you made during the refinement process. Including all code snippets where you made a change wrapped in python .

Key Points to Remember:
The bug or issue must always be resolved within the current GAU, as other units are either fully implemented and tested or placeholders that do not perform any computation.
Ensure the GAU is self-contained, so you won't need to adjust it later when working on other units.
The design must align with the original proposal and follow all instructions, templates, and format requirements.
Use a top-down approach: break down complex operations into smaller tasks where necessary and declare each of them as a child GAU. Do not make a single unit overly complex.
Remember your final goal is to refine the GAU in a way that enhances the overall design, ensuring both correctness and innovation.
'''

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage


import sys
sys.path.append('..')

import model_discovery.agents.prompts.prompts as P




DESIGNER_SYSTEM=P.GUE_DESIGNER_SYSTEM(GAB_BASE=GAB_BASE,GAU_BASE=GAU_BASE,GAU_TEMPLATE=GAU_TEMPLATE)
system=[{"type": "text", "text": DESIGNER_SYSTEM, "cache_control": {"type": "ephemeral"}}]

# messages = [
#     SystemMessage(system),
#     HumanMessage(TEST_INIT_DESIGN_INPUT)
# ]

message=[
    {
        'role': 'user',
        'content': [{
            "type": "text",
            "text": TEST_INIT_DESIGN_INPUT,
            "cache_control": {"type": "ephemeral"}
        }]
    }
]
extra_headers={"anthropic-beta": "prompt-caching-2024-07-31"}

SELECTIONS=['GPT2','RMSNorm','GatedMLP','MHA','RotaryPositionalEmbeddings']
lc_model = ChatAnthropic(
    model="claude-3-5-sonnet-20240620", temperature=0,
    extra_headers={"anthropic-beta": "prompt-caching-2024-07-31"}
)
structured_llm = lc_model.with_structured_output(P.gen_GUE_DESIGN_PROPOSAL(SELECTIONS).format,include_raw=True)
tools_args=structured_llm.dict()['first']['steps__']['raw']['kwargs']
        # tools[0]['cache_control']={"type": "ephemeral"}

RET=anthropic.Anthropic().messages.create(
    model="claude-3-5-sonnet-20240620", # model in config is ignored
    max_tokens=8192,
    messages=message, 
    temperature=0,
    system=system, # claude does not has system role, system prompt must be passed separately
    extra_headers=extra_headers,
    **tools_args
)


In [ ]:
RET.dict()['content']

In [ ]:
P.gen_GUE_DESIGN_PROPOSAL(SELECTIONS).format.model_validate(RET.content[0].input)


In [ ]:
RET.dict()

In [ ]:
structured_llm.dict()['first']['steps__']['raw']['kwargs']['tools'][0]['input_schema']

In [ ]:
Message(id='msg_01Nvch8XBwzwrrMBETHpch49', content=[TextBlock(text="Thank you for providing the seed design and references. I'll now propose a design improvement for the GPT2 model, focusing on modifying one of its GAUs.", type='text'), ToolUseBlock(id='toolu_01PpbFTQ6NDwJJCfj1cRDvFS', input={'selection': 'RotaryPositionalEmbeddings', 'modelname': 'GPT2-RoPE', 'proposal': "\n# GPT2 with Enhanced Rotary Position Embeddings (GPT2-RoPE)\n\n## Motivation\n\nThe current GPT2 model uses a basic implementation of Rotary Positional Embeddings (RoPE). While this is already an improvement over traditional positional encodings, there's room for enhancement, particularly in handling longer sequences and improving the model's ability to capture relative positions more effectively.\n\n## Problem Analysis\n\n1. **Limited sequence length handling**: The current implementation may not scale well to very long sequences, which is increasingly important in modern language models.\n2. **Lack of flexibility**: The existing RoPE implementation doesn't allow for easy adjustment of its properties, such as the rate of position information decay.\n3. **Potential for improved performance**: While RoPE is effective, recent research suggests that more sophisticated implementations can lead to better model performance, especially in tasks requiring understanding of long-range dependencies.\n\n## Core Idea and Philosophy\n\nWe propose to enhance the RotaryPositionalEmbeddings GAU by incorporating ideas from the RoFormer paper and the more recent developments in rotary embeddings. The core philosophy is to create a more flexible and powerful positional embedding system that can:\n\n1. Handle longer sequences more effectively\n2. Allow for fine-tuning of the position encoding properties\n3. Potentially improve the model's performance on tasks requiring long-range understanding\n\n## Design Plan\n\nWe will modify the RotaryPositionalEmbeddings GAU as follows:\n\n1. **Implement a more sophisticated RoPE class**: Based on the RoFormer paper and recent developments, we'll create a new RotaryEmbedding class that offers more flexibility and features.\n\n2. **Add support for learned frequencies**: This will allow the model to potentially learn better position representations for specific tasks or domains.\n\n3. **Implement interpolation factor**: This will enable the model to handle sequences longer than those seen during training more effectively.\n\n4. **Add support for XPos (Extrapolatable Position Embeddings)**: This will further enhance the model's ability to handle longer sequences and improve its understanding of relative positions.\n\n5. **Implement caching mechanism**: To improve efficiency, especially for autoregressive generation tasks.\n\n6. **Add support for different frequency types**: This will allow the embeddings to be optimized for different types of data (e.g., language, images).\n\nHere's a high-level overview of the proposed changes:\n\n```python\nclass RotaryEmbedding(nn.Module):\n    def __init__(\n        self,\n        dim,\n        custom_freqs = None,\n        freqs_for = 'lang',\n        theta = 10000,\n        max_freq = 10,\n        num_freqs = 1,\n        learned_freq = False,\n        use_xpos = False,\n        xpos_scale_base = 512,\n        interpolate_factor = 1.,\n        theta_rescale_factor = 1.,\n        seq_before_head_dim = False,\n        cache_if_possible = True\n    ):\n        super().__init__()\n        # Implementation details...\n\n    def rotate_queries_or_keys(self, t, seq_dim = None, offset = 0, scale = None):\n        # Implementation details...\n\n    def rotate_queries_and_keys(self, q, k, seq_dim = None):\n        # Implementation details...\n\n    def get_scale(self, t, seq_len = None, offset = 0):\n        # Implementation details...\n\n    def forward(self, t, seq_len = None, offset = 0):\n        # Implementation details...\n```\n\nThis new implementation will replace the current RotaryPositionalEmbeddings GAU in the GPT2 model.\n\n## Conclusion\n\nBy enhancing the RotaryPositionalEmbeddings GAU with these improvements, we expect to see several benefits:\n\n1. **Improved handling of long sequences**: The model should be able to maintain performance even on sequences longer than those seen during training.\n2. **Better capture of relative positions**: The enhanced RoPE implementation should allow the model to better understand and utilize relative position information.\n3. **Increased flexibility**: The new implementation allows for fine-tuning of various aspects of the positional embeddings, which can be optimized for specific tasks or domains.\n4. **Potential performance improvements**: These enhancements may lead to improved performance on tasks requiring long-range understanding and more nuanced use of positional information.\n\nThis modification maintains the overall structure of the GPT2 model while significantly enhancing its ability to handle and utilize positional information, potentially leading to improved performance across a wide range of natural language processing tasks.\n\n## References\n\n1. Su, J., Lu, Y., Pan, S., Wen, B., & Liu, Y. (2021). RoFormer: Enhanced Transformer with Rotary Position Embedding. arXiv preprint arXiv:2104.09864.\n2. Dao, T., & Gu, A. (2024). Transformers are SSMs: Generalized Models and Efficient Algorithms Through Structured State Space Duality. arXiv preprint arXiv:2401.04088.\n"}, name='GUE_DESIGN_PROPOSAL_format', type='tool_use')], model='claude-3-5-sonnet-20240620', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(input_tokens=16630, output_tokens=1325))

In [ ]:
content='Reflection on Feedback:\nThe reviewer\'s suggestions highlight the need for efficiency optimization, empirical validation, and thorough documentation. I\'ll address these points in the refined design.\n\nNew Analysis and Design:\n\nThe current GatedMLP design introduces a dynamic gating mechanism that allows the model to select between multiple activation functions. While this approach enhances adaptability, it may introduce computational overhead. To address this, we\'ll implement a more efficient gating mechanism that maintains the benefits of dynamic activation selection while reducing computational costs.\n\nUpdated Design Direction:\n1. Efficient Gating: Instead of applying all activation functions and then combining them, we\'ll use a lightweight gating network to predict weights for each activation function. This approach reduces the number of computations while still allowing dynamic selection.\n\n2. Sparse Activation: We\'ll introduce a sparsity factor in the gating mechanism, allowing the model to focus on the most relevant activation functions for each input.\n\n3. Adaptive Computation: We\'ll implement a mechanism to dynamically adjust the number of active gates based on the input complexity, further optimizing computational resources.\n\nHigh-level Pseudocode:\n```\nfunction GatedMLP(X):\n    hidden = linear_projection(X)\n    gate_weights = lightweight_gate_network(hidden)\n    sparse_gate_weights = apply_sparsity(gate_weights)\n    activated = sum(sparse_gate_weights[i] * activation[i](hidden) for i in range(num_activations))\n    output = linear_projection(activated)\n    return output\n```\n\nUpdated GAU Document:\nThe GatedMLP unit now incorporates an efficient dynamic gating mechanism that selects between multiple activation functions. Key features include:\n- Lightweight gating network for predicting activation weights\n- Sparse activation selection to focus on the most relevant functions\n- Adaptive computation based on input complexity\n- Maintains the same input-output interface as the original GatedMLP\n\nThis design aims to balance expressiveness and computational efficiency, allowing the model to adapt to various input patterns while managing resource usage effectively.\n\nImplementation:\n\n```python\n# gau.py\n\nimport torch\nimport torch.nn as nn\nimport torch.nn.functional as F\nfrom model_discovery.model.utils.modules import GAUBase, gau_test\n\nclass GatedMLP(GAUBase):\n    """\n    Generalized Autoregressive Block Unit with efficient dynamic gating\n    Input:  X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} \n    Output: Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of new intermediate variables to update the current Z}\n    """\n    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,\n                 device=None, dtype=None, hidden_features=None, out_features=None,\n                 bias=False, multiple_of=128, num_activations=3, sparsity_factor=0.5, **kwargs):\n        self.factory_kwargs = {\'device\': device, \'dtype\': dtype}\n        super().__init__(embed_dim, block_loc, kwarg_all)\n        \n        out_features = out_features if out_features is not None else embed_dim\n        hidden_features = hidden_features if hidden_features is not None else int(8 * embed_dim / 3)\n        hidden_features = (hidden_features + multiple_of - 1) // multiple_of * multiple_of\n        \n        self.fc1 = nn.Linear(embed_dim, hidden_features, bias=bias, **self.factory_kwargs)\n        self.fc2 = nn.Linear(hidden_features, out_features, bias=bias, **self.factory_kwargs)\n        \n        self.gate_network = nn.Sequential(\n            nn.Linear(hidden_features, 64, bias=False, **self.factory_kwargs),\n            nn.ReLU(),\n            nn.Linear(64, num_activations, bias=False, **self.factory_kwargs)\n        )\n        \n        self.activations = [F.relu, F.tanh, F.gelu]\n        self.sparsity_factor = sparsity_factor\n\n    def _forward(self, X, **Z):\n        hidden = self.fc1(X)\n        gate_weights = self.gate_network(hidden)\n        \n        # Apply softmax and sp'

print(content)

# Read from core library

In [1]:
import sys
sys.path.append('..')

import importlib
import model_discovery.model.composer 
# importlib.reload(model_discovery.model.composer)
from model_discovery.model.composer import GAUBase, GAUTree, check_tree_name, GABComposer
from model_discovery.model.utils.modules import GABBase
import model_discovery.utils as U

# dir='/home/junyanc/model_discovery/model_discovery/model/library/core/gpt2/units'
dir='C://ChengJunyan1//Research//model_discovery//model_discovery//model//library//core//gpt2//units'
tree=GAUTree.load_from_base(dir)

print(tree.view()[0])

c:\Users\ChengJunyan1\anaconda3\envs\modis\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\ChengJunyan1\.cache\huggingface\token
Login successful


In [ ]:
import inspect


gam_prompt_path = 'C://ChengJunyan1//Research//model_discovery//model_discovery//agents//prompts//gam_prompt.py'
gau_template_path = 'C://ChengJunyan1//Research//model_discovery//model_discovery//agents//prompts//gau_template.py'
GAM_TEMPLATE=open(gam_prompt_path).read()
GAU_TEMPLATE=open(gau_template_path).read()

GAU_BASE=inspect.getsource(GAUBase)
GAB_BASE=inspect.getsource(GABBase)
GAB_COMPOSER=inspect.getsource(GABComposer)


In [ ]:
print(GAM_TEMPLATE)

## Test 

In [59]:
import yaml
import inspect
import importlib
import os,sys
import importlib
import arxiv

sys.path.append('..')

from model_discovery.evolution import BuildEvolution
import model_discovery.utils as U
import requests
import time
from tqdm import tqdm

headers={"X-API-KEY": os.environ['S2_API_KEY']}
paper_detail='https://api.semanticscholar.org/graph/v1/paper/{paper_id}'


strparams=[
    f"evoname=test_evo_000",
    "scales=14M,31M,70M",
    "selection_ratio=0.25",
    "select_method=random",
    "design_budget=0",
]
evo_system = BuildEvolution(
    strparams=';'.join(strparams),
    do_cache=False,
    # cache_type='diskcache',
)

ptree = evo_system.ptree


2024-09-12:19:13:54,888 INFO     [model_agent.py:184] Agent name=`designer`, model_details={
    "model_name": "gpt-4o-2024-08-06",
    "max_output_tokens": 8192,
    "temperature": 0.3
}


Loaded tokenized dataset wikitext-2 from C:\ChengJunyan1\Research\model_discovery\data\wikitext-2\tokenized\meta-llama/Llama-2-7b-hf\2048


2024-09-12:19:13:56,847 INFO     [model_agent.py:184] Agent name=`debugger`, model_details={
    "model_name": "gpt-4o-mini",
    "max_output_tokens": 8192
}
2024-09-12:19:13:57,209 INFO     [model_agent.py:184] Agent name=`claude`, model_details={
    "model_name": "gpt-4o",
    "max_output_tokens": 8192
}


{'evoname': 'test_evo_000', 'scales': '14M,31M,70M', 'selection_ratio': '0.25', 'select_method': 'random', 'design_budget': '0'}
Evolution system initialized with scales: ['14M', '31M', '70M']
Current scale: 0
Budgets remaining: {'70M': 1, '31M': 4, '14M': 16}
Checkpoint directory: C:\ChengJunyan1\Research\model_discovery\ckpt\test_evo_000
gpt2 tree loaded


2024-09-12:19:13:58,936 INFO     [model_agent.py:184] Agent name=`designer`, model_details={
    "model_name": "gpt-4o-2024-08-06",
    "max_output_tokens": 8192,
    "temperature": 0.3
}


Loaded tokenized dataset wikitext-2 from C:\ChengJunyan1\Research\model_discovery\data\wikitext-2\tokenized\meta-llama/Llama-2-7b-hf\2048


2024-09-12:19:13:59,744 INFO     [model_agent.py:184] Agent name=`debugger`, model_details={
    "model_name": "gpt-4o-mini",
    "max_output_tokens": 8192
}
2024-09-12:19:14:00,132 INFO     [model_agent.py:184] Agent name=`claude`, model_details={
    "model_name": "gpt-4o",
    "max_output_tokens": 8192
}


In [60]:
import importlib
import sys
# sys.path.append('..')

import model_discovery.agents.search_utils
importlib.reload(model_discovery.agents.search_utils)

from model_discovery.agents.search_utils import SuperScholarSearcher, pwc_search_patched

sss=SuperScholarSearcher(ptree,evo_system.stream)



2024-09-12:19:14:02,041 INFO     [_client.py:1038] HTTP Request: GET https://api.openai.com/v1/vector_stores/vs_a2L6ajjzEki2N1krUFz3JZxz "HTTP/1.1 200 OK"
2024-09-12:19:14:02,213 INFO     [_client.py:1038] HTTP Request: GET https://api.openai.com/v1/vector_stores/vs_BDqAPFMyIPSrZkk657zgNiGY "HTTP/1.1 200 OK"
2024-09-12:19:14:02,409 INFO     [_client.py:1038] HTTP Request: GET https://api.openai.com/v1/vector_stores/vs_1lc8Tgkl6FTwOzmQ9TAK3ZsO "HTTP/1.1 200 OK"


In [61]:
# sss._load_libs()
# sss._load_texts()

In [110]:
import cohere


co = cohere.Client(os.environ['COHERE_API_KEY'])


In [112]:
_docs=[i.page_content for i in docs]

response = co.rerank(
    model="rerank-english-v3.0",
    query="What is the capital of the United States?",
    documents=_docs,
    top_n=3,
)
print(response)

2024-09-12:22:04:18,912 INFO     [_client.py:1038] HTTP Request: POST https://api.cohere.com/v1/rerank "HTTP/1.1 200 OK"


id='e8633431-293e-405d-82f3-4b47d524a89f' results=[RerankResponseResultsItem(document=None, index=5, relevance_score=0.001012286), RerankResponseResultsItem(document=None, index=12, relevance_score=0.000628784), RerankResponseResultsItem(document=None, index=14, relevance_score=0.00040448149)] meta=ApiMeta(api_version=ApiMetaApiVersion(version='1', is_deprecated=None, is_experimental=None), billed_units=ApiMetaBilledUnits(input_tokens=None, output_tokens=None, search_units=1.0, classifications=None), tokens=None, warnings=None)


In [118]:
indices=[i.index for i in response.results]
relevance_scores=[i.relevance_score for i in response.results]

In [120]:
_docs[indices]

TypeError: list indices must be integers or slices, not list

In [108]:
e=embedding.embed_documents([docs[0].page_content,docs[1].page_content])

2024-09-12:20:48:52,900 INFO     [_client.py:1038] HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [109]:
len(e)

2

In [101]:
docs = text_splitter.create_documents([sss.texts['3drpe']])
print(len(docs))
print(docs[1].page_content)

2024-09-12:20:03:49,202 INFO     [_client.py:1038] HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


20
The context size is $L$, and the chunk size is $c$. The vectors $\left[\boldsymbol{h}_{j, m}^{1}, \boldsymbol{h}_{j, m}^{2}\right]^{T}$ and $\left[-\boldsymbol{h}_{j, m}^{2}, \boldsymbol{h}_{j, m}^{1}\right]^{T}$ form an orthogonal basis, corresponding to the $|1\rangle$ and $|0\rangle$ states in Eq. 11. The components $\boldsymbol{h}_{j, m}^{1}$ and $\boldsymbol{h}_{j, m}^{2}$ represent the first and second dimensions of the state vector $\boldsymbol{h}_{j, m}$, which is the $m_{t h}$ token in the $j_{t h}$ chunk. i is the imaginary unit. $\boldsymbol{h}_{j, m}^{\perp}$ equals to $\left[-\boldsymbol{h}_{j, m}^{2}, \boldsymbol{h}_{j, m}^{1}\right]^{T}$, where $\boldsymbol{h}_{j, m}^{1} \in \mathbb{R}^{d / 2}$ and $\boldsymbol{h}_{j, m}^{2} \in \mathbb{R}^{d / 2}$ is the first and second halves of the state vector $\boldsymbol{h}_{j, m}$. In transformer-based LLMs, after applying position encoding to the state vectors from Query and Key, it is essential to compute their attention sco

In [102]:
print(docs[2].page_content)

0 3}$ | $\mathbf{7 . 1 0}$ | 8.09 | 8.12 | $\mathbf{2 . 7 2}$ | 2.93 | 2.89 | 3.05 |

Table 4: Results are evaluated in Perplexity on PG19 validation split. '*' denotes train-free, implementing 3D-RPE directly on the LLaMA2-7B-Base model without additional fine-tuning, utilizing a chunk size of $3 k$. The context length of $8 k$ is extended directly with 3D-RPE. Achieving $16 k$ and $32 k$ is accomplished through linear positional interpolation with chunks based on the $8 k$ context length. | MODELS | $4 k$ | $8 k$ | $16 k$ | $32 k$ |
| :--- | :---: | :---: | :---: | :---: |
| LLaMA2-7B-PI | 7.94 | 9.19 | 15.11 | $>10^{2}$ |
| LLaMA2-7B-NTK | 7.87 | 11.98 | 26.12 | 58.91 |
| LLaMA2-7B-Yarn | 7.87 | 8.06 | 9.82 | 11.74 |
| 3D-RPE-LLaMA2-7B* | 7.87 | $\mathbf{7 .


#### Retry when recover

In [122]:
import urllib
from tqdm import tqdm
import time

pdfs2_dir=U.pjoin(sss.files_dir,'pdfs2')

for i in tqdm(sss.lib2):
    if 'externalIds' in sss.lib2[i]:
        if 'ArXiv' in sss.lib2[i]['externalIds']:
            arxiv_id=sss.lib2[i]['externalIds']['ArXiv']
            path=U.pjoin(pdfs2_dir,f'{i}.pdf')
            arxiv_pdf=f'https://arxiv.org/pdf/{arxiv_id}'
            if U.pexists(path):
                continue
            try:
                pdf=urllib.request.urlretrieve(arxiv_pdf, path)
                time.sleep(0.01)
            except Exception as e:
                print(e)




  6%|▌         | 265/4573 [00:00<00:01, 2348.83it/s]

HTTP Error 404: Not Found


100%|██████████| 4573/4573 [10:54<00:00,  6.99it/s]  
